# CrewAI Multi-Agents Demo

# Installs

In [ ]:
!pip install -q crewai==0.28.8
!pip install -q crewai_tools==0.1.6
!pip install -q langchain_community==0.0.29
!pip install -q arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 871.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 15.3 MB/s

# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import BaseTool
from langchain.chat_models import ChatOpenAI
import arxiv
import requests
from dotenv import load_dotenv
import os
from IPython.display import Markdown

In [ ]:
%load_ext dotenv
%dotenv /content/drive/MyDrive/.env

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define API tools

In [ ]:
class FetchPapersTool(BaseTool):
    name: str = "Fetch AI Research Papers"
    description: str = "Fetch latest AI research papers from Arxiv based on a search query and optional author affiliations."

    def _run(self, keywords: list=["artificial intelligence"], affiliations: list = [], max_results=5) -> list:
        query = " OR ".join(keywords)  # Convert list into an OR-separated query
        if affiliations:
            affiliation_query = " OR ".join([f"au:{aff}" for aff in affiliations])
            query = f"({query}) AND ({affiliation_query})"

        search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.SubmittedDate)
        papers = []
        for result in search.results():
            papers.append({"title": result.title, "summary": result.summary, "url": result.entry_id})
        return papers

fetch_papers_tool = FetchPapersTool()

# Define Agents

In [ ]:
llm = ChatOpenAI(model_name='gpt-4o', api_key = OPENAI_API_KEY)

## 1. Research Agent

In [ ]:
retrieval_agent = Agent(
    role = "Paper Retrieval Agent",
    goal = "Retrieve the latest research papers from Arxiv on a given set of topics.",
    backstory = (
        "Uses Arxiv to explore the most recent and relevant papers "
        "given a set of keywords. This is a very up-to-date repository, "
        "which guarantees that we are accessing the cutting-edge contributions to the specified fields."
    ),
    tools=[fetch_papers_tool],
    llm=llm
)

/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 2. Summarization Agent

In [ ]:
summarization_agent = Agent(
    role="Summarization Agent",
    goal="Summarize research papers in layman's terms.",
    backstory = (
        "Digests the contents of the papers, no matter how technically complex. "
        "Extracts the key ideas so that they can be understood by someone interested in the field "
        "but not necessarily trained on the subject."
    ),
    llm=llm
)

## 3. *Influencer* Agent

In [ ]:
content_generation_agent = Agent(
    role = "Content Generation Agent",
    goal = "Creates LinkedIn posts and newsletter articles from research summaries.",
    backstory = (
        "Showing super-powers to engage readers, this influencer agent "
        "takes texts on scientific or technological subjects and builds "
        "engaging summaries in a newsletter format with a potential high impact."
    ),
    llm = llm
)

# Create Tasks

## 1. Research Task

In [ ]:
retrieval_task = Task(
    description="Fetch the latest research papers on the topics provided, "
                "focusing on the work of researchers "
                "from the institutions specified.",
    agent=retrieval_agent,
    expected_output= "A list of texts containing the contents of the most recent "
                     "and relevant papers on the topics specified.",
    input_variables=["keywords", "affiliations"]
)

## 2. Summarization Task

In [ ]:
summarization_task = Task(
    description="Summarize the key findings of the retrieved research",
    agent=summarization_agent,
    expected_output = (
        "A list of summaries distilling the contents of the "
        "technical articles in the input, 500 to 1000 words each."
    )
)

## 3. Content generation Task

In [ ]:
content_task = Task(
    description="Generate an engaging newsletter",
    agent=content_generation_agent,
    expected_output = "A 500 to 1000 word newsletter in Markdown format based on the provided summaries. "
                      "Written in an accessible and engaging language, it must visually appealing by using "
                      "bullets, bold letters, italic letters and hyperlinks to the sources.",
)

# Orchestrate Agents

In [ ]:
crew = Crew(
    agents=[retrieval_agent, summarization_agent, content_generation_agent],
    tasks=[retrieval_task, summarization_task, content_task],
    verbose = True
)

# Execute

In [ ]:
keywords = ["artificial intelligence", "deep learning", "machine learning", "LLMs", "neural networks", "agentic ai"]
affiliations = ["openai", "deepmind", "stanford", "microsoft"]
results = crew.kickoff(inputs={'keywords': keywords, 'affiliations': affiliations})

 [DEBUG]: == Working Agent: Paper Retrieval Agent
 [INFO]: == Starting Task: Fetch the latest research papers on the topics provided, focusing on the work of researchers from the institutions specified.
 

[{'title': 'L$^2$M: Mutual Information Scaling Law for Long-Context Language Modeling', 'summary': "We rigorously establish a bipartite mutual information scaling law in natural\nlanguage that governs long-range dependencies. This scaling law, which we show\nis distinct from and scales independently of the conventional two-point mutual\ninformation, is the key to understanding long-context language modeling. Using\nthis scaling law, we formulate the Long-context Language Modeling (L$^2$M)\ncondition, which relates a model's capacity for effective long context length\nmodeling to the scaling of its latent state size for storing past information.\nOur results are validated through experiments on both transformers and state\nspace models. This work establishes a theoretical foundation t

In [ ]:
display(Markdown(results))


# 📰 Cutting-Edge Insights in AI and Robotics

Welcome to this week's edition of our newsletter, where we dive into groundbreaking research shaping the future of AI and robotics. From novel scaling laws in language models to innovative approaches in scene flow estimation, let's explore these exciting advancements. 

## 📚 L$^2$M: A New Perspective on Long-Context Language Modeling

Understanding long pieces of text is a complex task for language models. The paper on **L$^2$M: Mutual Information Scaling Law for Long-Context Language Modeling** introduces a fresh approach to tackling this challenge. 

- **Concept**: Focuses on mutual information, a measure of dependency between variables.
- **Innovation**: Proposes a new scaling law to explain how models store and process lengthy sequences.
- **Outcome**: The Long-context Language Modeling (L$^2$M) condition links a model's capacity to manage extended texts to its latent memory size. 
- **Support**: Experiments with transformers and other models bolster this framework, paving the way for developing large models that handle long texts effectively.

*Explore the full paper [here](http://arxiv.org/abs/2503.04725v1).*

## 🎲 Bayesian Behavior in Large Language Models

Can Large Language Models (LLMs) learn like humans with limited data? The study **Enough Coin Flips Can Make LLMs Act Bayesian** investigates this intriguing question.

- **Focus**: Examines few-shot learning and its alignment with Bayesian principles.
- **Findings**: LLMs start with biased views but update beliefs Bayesianly with more examples.
- **Insight**: Even against explicit contradictory instructions, LLMs rely more on examples, showcasing a Bayesian-like learning approach.

*Read more about this study [here](http://arxiv.org/abs/2503.04722v1).*

## 🚀 Floxels: Advancing Scene Flow Estimation

The **Floxels** paper presents a leap forward in unsupervised scene flow estimation, crucial for robotics applications.

- **Challenge**: Existing methods face domain gaps and inefficiencies.
- **Solution**: A voxel grid-based model paired with a new multiframe loss formulation.
- **Advantage**: Floxels outperform current unsupervised methods in speed and accuracy, significantly reducing runtime while maintaining high benchmark performance.

*Discover the details [here](http://arxiv.org/abs/2503.04718v1).*

## 🌌 MIGHTEE: Unveiling Galactic Relationships

Exploring the cosmos, the **MIGHTEE** study delves into the relationship between spectral index, redshift, and radio luminosity.

- **Research**: Uses data from various telescopes to challenge existing assumptions.
- **Revelation**: The spectral index-redshift relationship varies across luminosity levels.
- **Caution**: Warns against relying solely on steep-spectrum selection for identifying high-redshift sources.

*Learn more about this cosmic exploration [here](http://arxiv.org/abs/2503.04717v1).*

## 🔧 Optimizing Hyperparameters in LLM Pretraining

The paper **Predictable Scale: Part I** identifies universal scaling laws for hyperparameters in language model pretraining.

- **Objective**: Enhance model deployment efficiency through optimal hyperparameter scaling.
- **Discovery**: Learning rates follow a power-law relationship with model and data sizes; batch sizes scale with data.
- **Tool**: A plug-and-play estimator aids in finding optimal settings, aligning closely with global optima.

*Check out the full research [here](http://arxiv.org/abs/2503.04715v1).*

---

These groundbreaking studies continue to push the boundaries of what is possible in AI and robotics. Stay tuned for more insights and innovations in our next edition!